In [28]:
import pandas as pd
import numpy as np
import nltk
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem.porter import *

In [2]:
df=pd.read_csv('got_scripts_breakdown.csv', sep=';')
df.head()

,Column1,Season,Episode,Sentence,Name,N_serie,N_Season,Emision Date
0,1,Season 1,winter is coming,What d’you expect? They’re savages One lot ste...,waymar royce,1,1,17/04/2011
1,2,Season 1,winter is coming,I’ve never seen wildlings do a thing like this...,will,1,1,17/04/2011
2,3,Season 1,winter is coming,How close did you get?,waymar royce,1,1,17/04/2011
3,4,Season 1,winter is coming,Close as any man would,will,1,1,17/04/2011
4,5,Season 1,winter is coming,We should head back to the wall,gared,1,1,17/04/2011


In [19]:
lines = df[["Sentence"]]
lines.reset_index(level=0, inplace=True)
documents = lines
#lines = ' '.join(lines)

print(len(documents))
print(documents[:5])

22510
   index                                           Sentence
0      0  What d’you expect? They’re savages One lot ste...
1      1  I’ve never seen wildlings do a thing like this...
2      2                             How close did you get?
3      3                             Close as any man would
4      4                    We should head back to the wall


In [56]:
def lemmatize_stemming(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

In [57]:
doc_sample = documents[documents['index'] == 4314].values[0][1]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))
    

original document: 
['Come', 'back', 'here!', 'Help', 'us,', 'boy!', 'A', 'man', 'can', 'fight!', 'Free', 'us']


 tokenized and lemmatized document: 
['come', 'help', 'boy', 'man', 'fight', 'free']


In [59]:
processed_docs = documents['Sentence'].map(preprocess)
processed_docs[:10]

0    [expect, savag, lot, steal, goat, lot, know, r...
1    [ve, see, wildl, thing, like, ve, see, thing, ...
2                                              [close]
3                                         [close, man]
4                                         [head, wall]
5                                     [dead, frighten]
6            [order, track, wildl, track, win, troubl]
7                          [think, ll, ask, die, hors]
8                                     [kill, children]
9    [good, thing, children, want, run, away, south...
Name: Sentence, dtype: object

In [60]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break


0 expect
1 goat
2 know
3 lot
4 piec
5 rip
6 savag
7 steal
8 life
9 like
10 see


In [61]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [63]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4314]

[(14, 1), (69, 1), (73, 1), (238, 1), (366, 1), (984, 1)]

In [70]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.3147120392948506),
 (1, 0.38853777590302263),
 (2, 0.15560655932944373),
 (3, 0.5936750852891856),
 (4, 0.3699308164512512),
 (5, 0.37194195201250624),
 (6, 0.31327559144079)]


In [71]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [72]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.063*"know" + 0.029*"father" + 0.026*"king" + 0.023*"dead" + 0.022*"lord" + 0.017*"hous" + 0.016*"lannist" + 0.016*"stark" + 0.015*"dont" + 0.014*"son"
Topic: 1 
Words: 0.038*"tell" + 0.023*"like" + 0.018*"oh" + 0.013*"your" + 0.013*"grace" + 0.013*"that" + 0.013*"cours" + 0.012*"get" + 0.011*"father" + 0.011*"come"
Topic: 2 
Words: 0.036*"like" + 0.032*"im" + 0.029*"go" + 0.029*"ye" + 0.025*"ladi" + 0.017*"time" + 0.016*"ll" + 0.014*"know" + 0.013*"queen" + 0.012*"trust"
Topic: 3 
Words: 0.027*"think" + 0.019*"walk" + 0.018*"take" + 0.017*"she" + 0.016*"happen" + 0.014*"maester" + 0.014*"lord" + 0.013*"meet" + 0.013*"queen" + 0.012*"command"
Topic: 4 
Words: 0.057*"come" + 0.030*"look" + 0.018*"turn" + 0.017*"stop" + 0.016*"let" + 0.016*"want" + 0.015*"ser" + 0.013*"mean" + 0.013*"stark" + 0.012*"hodor"
Topic: 5 
Words: 0.027*"need" + 0.024*"help" + 0.017*"dragon" + 0.017*"die" + 0.017*"littl" + 0.015*"look" + 0.014*"daeneri" + 0.013*"want" + 0.012*"im" + 0.011*"ve"


In [74]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWord: {}'.format(idx, topic))

Topic: 0 
Word: 0.024*"ladi" + 0.020*"father" + 0.017*"king" + 0.015*"boy" + 0.015*"say" + 0.013*"trust" + 0.011*"better" + 0.011*"rememb" + 0.010*"kill" + 0.009*"littl"
Topic: 1 
Word: 0.024*"think" + 0.023*"like" + 0.017*"dont" + 0.015*"im" + 0.014*"mean" + 0.014*"kill" + 0.014*"mother" + 0.012*"sorri" + 0.011*"brother" + 0.011*"want"
Topic: 2 
Word: 0.056*"ye" + 0.036*"lord" + 0.036*"grace" + 0.019*"hodor" + 0.014*"good" + 0.013*"that" + 0.009*"what" + 0.009*"there" + 0.008*"son" + 0.007*"command"
Topic: 3 
Word: 0.033*"want" + 0.022*"help" + 0.014*"know" + 0.014*"queen" + 0.011*"love" + 0.011*"arya" + 0.009*"hurt" + 0.008*"sam" + 0.008*"wait" + 0.008*"see"
Topic: 4 
Word: 0.035*"come" + 0.019*"leav" + 0.015*"time" + 0.012*"lord" + 0.011*"jon" + 0.011*"north" + 0.010*"want" + 0.009*"oh" + 0.008*"command" + 0.008*"god"
Topic: 5 
Word: 0.043*"know" + 0.016*"fuck" + 0.016*"men" + 0.013*"drink" + 0.013*"way" + 0.012*"talk" + 0.010*"thank" + 0.010*"didnt" + 0.009*"love" + 0.009*"like"
To

In [75]:
processed_docs[4314]

['come', 'help', 'boy', 'man', 'fight', 'free']

In [76]:
for index, score in sorted(lda_model[bow_corpus[4314]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5517072081565857	 
Topic: 0.027*"need" + 0.024*"help" + 0.017*"dragon" + 0.017*"die" + 0.017*"littl" + 0.015*"look" + 0.014*"daeneri" + 0.013*"want" + 0.012*"im" + 0.011*"ve"

Score: 0.33396920561790466	 
Topic: 0.057*"come" + 0.030*"look" + 0.018*"turn" + 0.017*"stop" + 0.016*"let" + 0.016*"want" + 0.015*"ser" + 0.013*"mean" + 0.013*"stark" + 0.012*"hodor"

Score: 0.014292757026851177	 
Topic: 0.043*"kill" + 0.032*"want" + 0.025*"good" + 0.019*"boy" + 0.015*"sword" + 0.014*"men" + 0.014*"way" + 0.013*"fuck" + 0.013*"stand" + 0.013*"jaim"

Score: 0.014292537234723568	 
Topic: 0.036*"like" + 0.032*"im" + 0.029*"go" + 0.029*"ye" + 0.025*"ladi" + 0.017*"time" + 0.016*"ll" + 0.014*"know" + 0.013*"queen" + 0.012*"trust"

Score: 0.014292149804532528	 
Topic: 0.030*"your" + 0.030*"right" + 0.018*"leav" + 0.017*"men" + 0.015*"know" + 0.014*"man" + 0.014*"castl" + 0.014*"wall" + 0.013*"friend" + 0.012*"better"

Score: 0.01429145596921444	 
Topic: 0.029*"lord" + 0.028*"grace" + 0.024*"

In [77]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4314]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6113854646682739	 
Topic: 0.033*"want" + 0.022*"help" + 0.014*"know" + 0.014*"queen" + 0.011*"love" + 0.011*"arya" + 0.009*"hurt" + 0.008*"sam" + 0.008*"wait" + 0.008*"see"

Score: 0.2742900848388672	 
Topic: 0.024*"ladi" + 0.020*"father" + 0.017*"king" + 0.015*"boy" + 0.015*"say" + 0.013*"trust" + 0.011*"better" + 0.011*"rememb" + 0.010*"kill" + 0.009*"littl"

Score: 0.014293383806943893	 
Topic: 0.020*"your" + 0.018*"die" + 0.015*"fight" + 0.015*"stark" + 0.014*"stop" + 0.014*"let" + 0.010*"right" + 0.010*"watch" + 0.009*"sansa" + 0.009*"lie"

Score: 0.01429283432662487	 
Topic: 0.035*"come" + 0.019*"leav" + 0.015*"time" + 0.012*"lord" + 0.011*"jon" + 0.011*"north" + 0.010*"want" + 0.009*"oh" + 0.008*"command" + 0.008*"god"

Score: 0.014290866442024708	 
Topic: 0.043*"know" + 0.016*"fuck" + 0.016*"men" + 0.013*"drink" + 0.013*"way" + 0.012*"talk" + 0.010*"thank" + 0.010*"didnt" + 0.009*"love" + 0.009*"like"

Score: 0.014290081337094307	 
Topic: 0.024*"think" + 0.023*"like" 